In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
import sympy
import mpmath
import pickle

#Create symbolic variables to solve equation numerically.
k, p, nn = sympy.symbols('k p nn')

<p style='text-align: justify;'>Spherical cap harmonic analysis (SCHA) was developed by Haines (1985) to model the field over a small portion of the globe. The basis functions of SHA and SCHA consist of associated Legendre functions in colatitude, trigonometric functions in longitude, and powers of radial distance (Haines, 1988). The difference is the associated Legendre functions of SHA have integral degrees, while for SCHA they have non-integral degrees. SCHA’s domain of interest is shown in following figure, and is defined by the half-angle of the spherical cap $\theta_0$  and the radial distances between $r = a$ and $r = b$.</p>

<img src="img/section.png" width="400"/>

<p style='text-align: justify;'>While the spherical harmonic functions of the SHA are orthogonal over the entire globe, they are orthogonal over the cap in SCHA. In SCHA, the boundary conditions must be applied to the edge of the cap in order to solve Laplace’s equation for the scalar magnetic potential. The boundary conditions require the continuity of the potential in longitude, regularity at the spherical cap pole, and both the potential and its derivative with respect to the colatitude at the cap boundary (Torta, 2019). In the SCHA model, the scalar magnetic potential $U$ at $\theta_0$ or at the edge of the spherical cap and its derivative in accordance with $\theta$ must be arbitrary</p>

\begin{align}U(r,\theta_0,\varphi_T )=f(r,\varphi_T )\end{align}

\begin{align}\frac{\partial U(r,\theta_0,\varphi_T )}{\partial\theta_T}=g(r,\varphi_T )\end{align}

<p style='text-align: justify;'>where $f$ and $g$ are arbitrary functions. The longitude and colatitude, $(θ_T,φ_T)$, in these boundary conditions and the SCHA calculations are in the spherical cap reference frame. As a result, it is essential to transform the site locations and the magnetic field components into this spherical cap reference frame.</p>

<p style='text-align: justify;'>A SCHA model has much the same form as for SHA, but $n$ no longer has to be integral. SCHA for the internal field is based upon the following expression</p>

\begin{align}U(r,\theta,\varphi)=a\sum^{\infty}_{m=0}\sum^{\infty}_{k=m}\left(\frac{a}{r}\right)^{n_k(m)+1} \big(g_k^m \; \cos\,(m\varphi_T)+h_k^m \; \sin\,(m\varphi_T)\big) \; P^m_{nk(m)} (\cos\,\theta_T)\end{align}

<p style='text-align: justify;'>This is similar to the SHA form; the difference is that now the associated Legendre polynomials are real and integral order ($m$) but not necessarily of integral degree ($n_k(m)$). The Gauss coefficients no longer describe a best fit to the geocentric dipole, quadrupole, and so on as in SHA, although they still form the basis of the model field. The coefficients are determined using the same methodology as SHA. The spatial truncation index ($k$) and order ($m$) in SCHA describe the sectioning over the spherical cap. This division defines the resolution of the features that can be mapped using SCHA (Fiori, 2020).</p>

<p style='text-align: justify;'>The Schmidt normalised Associated Legendre Polynomials in the solution above are</p>

\begin{align}P_n^m (\cos\,\theta)=\sum^{\infty}_{k=0} \; A_k (m,n) \left\{\frac{1-\cos\,\theta_T}{2}\right\}^k\end{align}

with

\begin{align}A_0 (m,n)=K(m,n) \; \sin^m\theta_T\end{align}

\begin{align}A_k (m,n)=\frac{(k+m-1)(k+m)-n(n+1)}{k(k+m)} \; A_{k-1} (m,n)\end{align}

In [117]:
# Define function to calculate Associate Legendre polynomials.
# The function is splitted into two kinds. The first function is used to define Associate Legendre equation where the n will be solved later.
#The second function is used to calculate the Associate Legendre polynomials where the n is already determined.
def assoc_Leg_poly(m, n, Kmn, thetaT, kmax):
    # Calculate Associate Legendre Polynomials. In this function, the value of n
    # is already determined
    A = np.zeros((kmax+1,1))
    for j in range(0,kmax+1):
        if j==0:
            A[j] = (Kmn*(np.sin(thetaT))**m)
        else:
            A[j] = A[j-1]*(((j+m-1)*(j+m)-n*(n+1))/(j*(j+m)))

    return A

In [118]:
def assoc_Leg_equation(m, n, Kmn, thetaT, kmax):
    # This function return Associate Legendre Polynomials equation
    for j in range(0,kmax+1):
        if j==0:
            A0 = (Kmn*(np.sin(thetaT))**m)
            P = A0*(((1-np.cos(thetaT))/2)**0)
            dP = A0*0*(((1-np.cos(thetaT))/2)**(0-1)/2)*np.sin(thetaT)+A0*\
                (((1-np.cos(thetaT))/2)**0)*m*np.cos(thetaT)/np.sin(thetaT)
        else:
            Ak = A0*mpmath.nprod(lambda k: (((k+m-1)*(k+m)-n*(n+1))/(k*(k+m))),[1,j])
            P = P + Ak*(((1-np.cos(thetaT))/2)**j)
            dP = dP + j*Ak*(((1-np.cos(thetaT))/2)**(j-1)/2)*np.sin(thetaT)\
                +Ak*(((1-np.cos(thetaT))/2)**j)*m*np.cos(thetaT)/      \
                np.sin(thetaT)
                
    return P, dP

<p style='text-align: justify;'>In this case, $K(m,n)$ is a normalising factor and obtained by</p>

\begin{align}K_n^m=1 \quad if \quad m=0\end{align}

\begin{align}K_n^m=\frac{2^{-m}}{\sqrt{m\pi}} \left (\frac{n+m}{n-m} \right)^{\frac{n}{4}+\frac{1}{4}} P^{\frac{m}{2}} \; exp⁡ \; (e_1+e_2) \quad if \quad m>0\end{align}

<p style='text-align: justify;'>Within this $P$ and $e_1$, $e_2$ can be obtained from</p>

\begin{align}P=\left ( \frac{n}{m} \right) ^2-1\end{align}

\begin{align}e_1=-\frac{1}{12m} \left(1+\frac{1}{P} \right)\end{align}

\begin{align}e_2=\frac{1}{360m^3} \left(1+ \frac{3}{P^2} + \frac{4}{P^3} \right)\end{align}

In [119]:
def norm_factor(n, m):
    #Define function to calculate normalising factor
    if m==0:
        Kmn = 1
    elif m>0:
        P = ((n/m)**2)-1
        e1 = -(1/(12*m))*(1+1/P)
        e2 = (1/(360*m**3))*(1+(3/P**2)+(4/P**3))
        Kmn = (2**(-m))/(np.sqrt(m*np.pi))*(((n+m)/(n-m))**((n/2)+(1/4)))*(P**(m/2))*np.exp(e1+e2)
            
    return Kmn

<p style='text-align: justify;'>For any combination of $(m,k)$, the Associated Legendre Polynomials are dependent on degree, $n_k(m)$, and colatitude within the cap, $\theta_T$. Also, the boundary conditions are equivalent to</p>

\begin{align}P_{n_k(m)}^m \; (\cos\, \theta_0)=0, \quad for \quad k-m=odd\end{align}

\begin{align}\frac{\partial P_{n_k(m)}^m \; (\cos⁡\,\theta_0)}{\partial \theta_T}=0, \quad for \quad k-m=even\end{align}

<p style='text-align: justify;'>Solving numerically for all $n_k(m)$ that satisfy one of equations above must be done in determining the non-integral degrees of the spherical harmonic functions in SCHA. Any $n_k(m)$ can satisfy only one of the boundary conditions. Thus, the solution comprises of two infinite sets of spherical harmonic functions which are mutually orthogonal within itself but not orthogonal to the other.</p>

In [120]:
def solve(function):
    # Range for guessing nonlinear least-square solution
    initial_guess = 80
    # Solve a nonlinear least-squares problem with bounds on the variables
    solution = least_squares(function,np.arange(0,initial_guess)).x
    # Filter negative roots
    solution = [item for item in solution if item >= 0]
    # Round the solution
    solution = np.sort(list(set(np.round(solution,4))))
    
    return solution

<p style='text-align: justify;'>The SCHA technique must be used with special care, especially when the cap is small. The result of the SCHA technique deteriorates as the cap becomes smaller. This is because a high degree expansion is needed in order to include large wavelengths over a small-cap, and can cause a problem if not enough data are available is (Verbanac, 2007). As a rule, the cap region must be at least coincident with the spatial content of the features of the field being represented (Torta, 2019). This problem has been solved by Torta et al. (1992) by increasing the cap half-angle to extend the wavelength content of the basis functions artificially. Nevertheless, beyond $\theta=20^{\circ}$, the SCHA was relatively insensitive to the chosen cap size (Düzgit & Malin, 2000).</p>

In [121]:
# Define the spherical cap size
theta = 30
# Define the maximum value of k for calculating A
kmax = 24
# Define the maximum value of m for calculating nk
mmax = 7

<p style='text-align: justify;'>Define an empty matrix of $n_k$, Legendre polynomials, and Schmidt-normalised functions so that the size is constant. We use symsum or symbolic summaries because the size of $n_k$ and $A$ is expanded linearly based on the index value ($m_{max}$).</p>

In [122]:
n = np.zeros(int(mpmath.nsum(lambda p: p,[1,mmax+1])))
A = np.zeros((int(mpmath.nsum(lambda p: p,[1,mmax+1])),kmax+1))
Kmn_all = np.zeros(int(mpmath.nsum(lambda p: p,[1,mmax+1])))
theta0 = np.deg2rad(theta)
# Index to determine first calculation
idx = 0

<p style='text-align: justify;'>The SCHA non-integer degree $n_k(m)$ must be solved numerically. Four decimal places are used to speed up the calculations as has been done by Haines (1985). A slight error in $n$ only affects the orthogonality of the relevant basis functions of the solutions. The decimal place required in the $n_k(m)$ depends on the half angle $\theta_0$ of the spherical cap. Fewer decimal places are required to obtain large $n_k(m)$ at high values of $k$ in order to maintain the orthogonality of the functions to an acceptable accuracy (Haines, 1988).</p>

In [123]:
for mm in range(mmax+1):
    idx = idx+1
    
    # For each k
    for m in range(mm,-1,-1):
        kk = mm
        
        # Index for nk and A position on matrix using Floyd's Triangle
        # 1
        # 3 2
        # 6 5 4
        # .......
        id = int((mm*(mm+1)/2)+m)
        
        # Index to define nk position from each numeric solving. It is
        # needed because each numeric solve have multiple solution.
        idy = int(np.floor((mm-m)/2))
        
        # P and dP for m=0
        if m==0:
            # Kmn=1 for m=0
            Kmn = norm_factor(0, m)
            # Calculate P and dP for k=0
            P = assoc_Leg_equation(m, nn, Kmn, theta0, kmax)[0]
            dP = assoc_Leg_equation(m, nn, Kmn, theta0, kmax)[1]

            # Special case for first nk calculation. We use dP=0 to get nk but use P=0 for the next calculation
            if idx==1:
                # Solve equation numerically using search interval as [0 Inf] to return nonnegative solutions
                eq_P = solve(sympy.lambdify(nn,P))[0]
                eq_dP = solve(sympy.lambdify(nn,dP))[0]
                # Join each nk into one matrix
                n[id] = eq_dP
                # Use P=0 solution for next calculation
                nk = eq_P
                # Calculate A using nk
                An = assoc_Leg_poly(m, eq_dP, Kmn, theta0, kmax) 
                # Join A and Kmn into one matrix
                A[id,:] = np.transpose(An)
                Kmn_all[id] = Kmn
                
            # For next calculation but if k-m is odd, we only use P=0
            elif idx!=1 and np.mod(kk-m,2)==1:
                # Solve equation numerically using search interval as [0 Inf] to return nonnegative solutions
                eq_P = solve(sympy.lambdify(nn,P))
                nk = eq_P[idy]
                # Join each nk into one matrix
                n[id] = nk
                # Calculate A using nk
                An = assoc_Leg_poly(m, nk, Kmn, theta0, kmax)
                # Join A and Kmn into one matrix
                A[id,:] = np.transpose(An)
                Kmn_all[id] = Kmn
                
            # For next calculation but if k-m is even, we only use dP=0
            elif idx!=1 and np.mod(kk-m,2)!=1:
                # Solve equation numerically using search interval as [0 Inf] to return nonnegative solutions
                eq_dP = solve(sympy.lambdify(nn,dP))
                nk = eq_dP[idy]
                # Join each nk into one matrix
                n[id] = nk
                # Calculate A using nk
                An = assoc_Leg_poly(m, nk, Kmn, theta0, kmax)
                # Join A and Kmn into one matrix
                A[id,:] = np.transpose(An)
                Kmn_all[id] = Kmn
                
        # P and dP for m=1 forward
        else:
            # Calculate Kmn using m and defined nk
            Kmn = norm_factor(nk, m)
            # Calculate P and dP for k=0
            P = assoc_Leg_equation(m, nn, Kmn, theta0, kmax)[0]
            dP = assoc_Leg_equation(m, nn, Kmn, theta0, kmax)[1]
                        
            # If k-m is odd, we only use P=0
            if np.mod(kk-m,2)==1:
                # Solve equation numerically using search interval as [0 Inf] to return nonnegative solutions
                eq_P = solve(sympy.lambdify(nn,P))
                nk = eq_P[idy]
            # If k-m is even, we only use dP=0
            elif np.mod(kk-m,2)!=1:
                # Solve equation numerically using search interval as [0 Inf] to return nonnegative solutions
                eq_dP = solve(sympy.lambdify(nn,dP))
                nk = eq_dP[idy]
            # Join each nk into one matrix
            n[id] = nk
            # Calculate Kmn using m and defined nk
            Kmn = norm_factor(nk, m)
            # Calculate A using nk
            An = assoc_Leg_poly(m, nk, Kmn, theta0, kmax)
            # Join A and Kmn into one matrix
            A[id,:] = np.transpose(An)
            Kmn_all[id] = Kmn

<p style='text-align: justify;'>Those values for which $k-m$ is odd (the second, fourth, sixth, … values) are the values of $n$ for which the function itself is zero at $\theta=30°$. Correspondingly, those values for which $k-m$ is even (the first, third, fifth, … values) are the values of $n$ is  for which the derivative of the function is zero at $\theta=30^{\circ}$. The $P_n^m (\cos \, \theta)$ and $n_k(m)$ are the functions of the spherical cap size. Therefore, these values will be similar for all regions as long as the spherical cap size is the same.</p>

In [129]:
nk_round = [np.float('%.4f' % elem) for elem in n]
with open('data\\output_'+str(mmax)+'_'+str(theta)+'.pkl', 'wb') as f:
    pickle.dump([nk_round,A,Kmn_all],f)
    
# Print the results of nkm
df_out = pd.DataFrame(index=range(mmax+1), columns=range(mmax+1))
selected_index = 0
rows = mmax+1
stop = 2
for i in range(rows):
    for j in range(1, stop):
        df_out[j-1][df_out.index==i] = '%.4f' % nk_round[selected_index]
        selected_index += 1
    stop += 1

print('Non integer degrees for spherical cap = '+str(theta)+'\u00b0 with kmax = '+str(mmax)+'\nThe columns show the order m, while the rows show the index k\n')
df_out.replace(np.nan, '', regex=True)

Non integer degrees for spherical cap = 30° with kmax = 7
The columns show the order m, while the rows show the index k



,0,1,2,3,4,5,6,7
0,0.0000,,,,,,,
1,4.0837,3.1196,,,,,,
2,6.8354,6.8354,5.4928,,,,,
3,10.0386,9.7121,9.3733,7.7524,,,,
4,12.9083,12.9083,12.3720,11.8074,9.9589,,,
5,16.0248,15.8215,15.6154,14.9180,14.1778,12.1334,,
6,18.9364,18.9364,18.5830,18.2219,17.3910,16.5041,14.2861,
7,22.0183,21.8702,21.7210,21.2461,20.7588,19.8121,18.7979,16.4228
